## Analysis of promotor and 3'UTR effect on expression

### Load libraries and functions

In [ ]:
import fcsparser as fcs
import os
import pandas as pd
import warnings
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
sns.set(style ="whitegrid")
sns.set_context("poster")
warnings.filterwarnings("once")

#functions for distribution plot
def multiple_dist_plot (dfx, FP, start):
    
    sns.set(style="white", font_scale=1, font="Arial", rc={"axes.facecolor": (0, 0, 0, 0)})
    sns.set_context("notebook")
    order = list(dfx.groupby(by=["Info"])[FP].aggregate(np.median).sort_values().iloc[::].index)
    if isinstance(start, str)==True:
        pal = sns.color_palette( start, len(order))
    else:
        pal = sns.cubehelix_palette(len(order), start=start)
    #pal.reverse()    
    #pal_order=[]
    #original_order=list(dfx.Info.unique())
    #for i in np.arange(len(order)):
    #    pal_order.append(order.index(original_order[i]))
    #pal=np.array(pal)
    #pal=pal[pal_order]
    
    g = sns.FacetGrid(dfx, row="Info", hue="Info", aspect=10, size=.5, palette=pal)#, row_order=order)

    # Draw the densities in a few steps
    #g.map(sns.distplot, FP)
    g.map(sns.kdeplot, FP, clip_on=False, shade=True, alpha=1, lw=1.5, bw=.2, color='gray')
    g.map(sns.kdeplot, FP, clip_on=False, color="w", lw=2, bw=.2)
    g.map(plt.axhline, y=0, lw=2, clip_on=False, color ='gray')

    # Define and use a simple function to label the plot in axes coordinates
    def label(x, color, label):
        ax = plt.gca()
        #ax.text(0, .2, [label,np.round(np.median(x), decimals=2)], fontweight="bold", color=color, 
                #ha="left", va="center", transform=ax.transAxes)
        ax.text(0, .2, label, fontweight="bold", color='gray', 
                ha="left", va="center", transform=ax.transAxes)

    g.map(label, FP)

    # Set the subplots to overlap
    g.fig.subplots_adjust(hspace=-0.25)

    # Remove axes details that don't play will with overlap
    g.set_titles("")
    g.set(yticks=[])
    g.despine(bottom=True, left=True)

### Load files and metadata

In [2]:
#folder with fcs files in subfolders
folder="C:\\FCS_experiment\\"

ffolderlist = [f for f in os.listdir(folder) if os.path.isdir(os.path.join(folder, f))]


df=pd.DataFrame()
aa=1

#load all fcs files
for fdn in ffolderlist:
    ffilelist = os.listdir(folder+fdn)
    print (fdn)
    plate = fdn.split("_")[2]
    run = fdn.split("_")[3]
    for fn in ffilelist:
        filename, file_ext = os.path.splitext(fn)
        if file_ext ==".fcs":
            path = folder + fdn + "//" + fn
            meta, df1 = fcs.parse(path, meta_data_only=False, reformat_meta=True)
            df1["WellName"]=fn.split("_")[3].split(".")[0]
            df1["WellNumber"]= aa
            df1["Plate"] =plate
            df1["Run"] = run
            df=df.append(df1)
            aa=aa+1

# metadata file
meta = "C:\\metadata.csv"
df1=pd.read_csv(meta, names=["WellName", "Name", "ExperimentName", "Info", "Plate"], dtype= object)

dfnew=df1.merge(df, on=["WellName", "Plate"])


3T3_PromTerm_Plate1_Run3


### remove zeros, gate live cells and log data

In [ ]:
#remove zeros
dfnew= dfnew[dfnew > 0]

#remove doublets
dfnew["doublet"] = dfnew["FSC-A"] / dfnew["FSC-H"]
df2=dfnew[(dfnew.doublet < 2)& (dfnew.doublet > 1.25)]
df2=dfnew

#remove small events
df2 =df2[df2["FSC-A"]>0E5]
df2 =df2[(df2["SSC-A"]<2E5)]

#show data for first well
g = sns.jointplot("FSC-A", "SSC-A", data=df2[df2["WellNumber"]==1], s=1)#.plot_joint(sns.kdeplot, zorder=0, n_levels=6);
plt.show()

#log data
for col in df2.columns:
    if df2[col].dtype=="float32":
        df2["log"+ col]=df2[col].apply(math.log10)


### select experiment and gate cells that express circuit

In [67]:
df_Term=df2[df2["ExperimentName"]=="Promotors"]
df_Term = df_Term[df_Term["logPE-CF594-A"]>3.5]
df_Term["normFITC-A"] = df_Term["FITC-A"]/df_Term["PE-CF594-A"]

### plot distributions

In [ ]:
   
FP = "logFITC-A"
start = "Greens_d"
multiple_dist_plot (dfTerm, FP, start)
#plt.savefig('D:\\Box Sync\\GoogleDrive\\UCSF\\Data\\2018\\FACS\\Terminators_A\\preGFPdist.png', format='png', dpi=300)
plt.show()


FP = "logPE-CF594-A"
start = "Reds_d"
multiple_dist_plot (dfTerm, FP, start)
#plt.savefig('D:\\Box Sync\\GoogleDrive\\UCSF\\Data\\2018\\FACS\\Terminators_A\\RFPdist.png', format='png', dpi=300)
plt.show()
        
FP = "logAPC-Cy7-A"
start = "Blues_d"
multiple_dist_plot (dfTerm, FP, start)
#plt.savefig('D:\\Box Sync\\GoogleDrive\\UCSF\\Data\\2018\\FACS\\Terminators_A\\RFPdist.png', format='png', dpi=300)
plt.show()


### remove wells with few cells

In [ ]:
df_Term_ind2 = df_Term.groupby(['Info', 'WellName']).count()
df_Term_ind2 = df_Term_ind2.reset_index()
excluded_wells=df_Term_ind2.loc[df_Term_ind2["FITC-A"]<50,"WellName"]


df_Term_ind = df_Term.groupby(['Info', 'WellName']).mean()
df_Term_ind = df_Term_ind.reset_index()
df_Term_ind = df_Term_ind[~df_Term_ind['WellName'].isin(excluded_wells)]

### normalize FITC signal and plot mean

In [ ]:
df_Term_ind['normnormFITC-A'] = df_Term_ind['normFITC-A']/np.mean(df_Term_ind.loc[df_Term_ind_1['Info']=='Spacer','normFITC-A'])
df_Term_ind['lognormnormFITC-A'] = df_Term_ind['normnormFITC-A'].apply(math.log10)
df_Term_ind = df_Term_ind_1.sort_values('normnormFITC-A')
df_Term_ind_g = df_Term_ind.groupby("Info")['lognormnormFITC-A'].mean().sort_values()

FP = "normnormFITC-A"
start = "gray"
multiple_dist_plot (df_Term_ind_1, FP, start)
plt.savefig(folder + 'dist.eps', type = 'eps')
plt.savefig(folder + 'dist.png', format='png', dpi=300)
plt.show()

fig=plt.figure(figsize=[10,10])
g2 = sns.barplot(x="normnormFITC-A", y="Info", data=df_Term_ind, color='gray', ci = None, order = df_Term_ind_g.index)
g2 = sns.stripplot(x="normnormFITC-A", y="Info", data=df_Term_ind, color='Black', order = df_Term_ind_g.index)
sns.despine(left=True, bottom= True)
plt.savefig(folder + 'bar.eps', type = 'eps')
plt.savefig(folder + 'bar.png', format='png', dpi=300)
plt.show()

df_Term_ind_1.to_csv(folder + 'data.csv')
